In [121]:
import pandas as pd 
from typing import Callable
import numpy as np
from scipy.fft import fft
from category_encoders import OneHotEncoder

In [120]:
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 862.9 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.9/233.9 kB 1.1 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 1.3 MB/s eta 0:00:0000:0100:01m


In [149]:
class FeatureEngine:
    def __init__(self,):
        self.training_set = pd.read_csv("cleaned_data/train_data.csv")
        self.validation_set = pd.read_csv("cleaned_data/validation_data.csv")
        self.test_set = pd.read_csv("cleaned_data/test_data.csv")
        self.demographic_set = pd.read_csv("Data Tables/HScreening.txt", delimiter = '|')
        #self.time_series = pd.read_csv("Experimental_Notebooks/resampled_day.csv")
        self.aggregate_window(self.tar, "tar")
        #self.aggregate_window(self.tbr, "tbr")
        #self.aggregate_window(self.rolling_mean, "mean")
        #self.aggregate_window(self.rolling_deviation, "std")
        #self.aggregate_window(self.tir, "tir")
        #self.aggregate_window(self.fft, "fft")
        #self.add_demographics()


    
    def fft(self, arr:np.ndarray) -> np.ndarray:
        return fft(arr)
    
    def rolling_deviation(self, arr:np.ndarray) -> np.ndarray:
        return np.std(arr)
    
    def tar(self, arr: np.ndarray) -> np.ndarray:
        mask = arr > 180
        return np.mean(mask)
    
    def tir(self, arr: np.ndarray) -> np.ndarray:
        mask = (arr >= 70) & (arr <= 180)
        return np.mean(mask)
    
    def tbr(self, arr: np.ndarray) -> np.ndarray:
        mask = arr < 70
        return np.mean(mask)
    
    def rolling_mean(self, arr: np.ndarray) -> np.ndarray:
        return np.mean(arr)

    def add_demographics(self) -> None:
        """
        Merge selected demographic features into the training, validation, and test sets.
        Encode them using one-hot encoding.
        """
        # Columns to merge
        columns_to_merge = [
            'PtID', 'Gender', 'Ethnicity', 'Race', 'SHMostRec', 'SHNumLast12Mon', 'DKAMostRec', 'DKANumLast12Mon',
            'OthGlucLowerMed', 'Weight', 'Height', 'PEAbnormal'
        ]

        # Filter the demographic set to include only the necessary columns
        demographics = self.demographic_set[columns_to_merge]

        # Prepare the encoder
        encoder = OneHotEncoder(cols=['Gender', 'Ethnicity', 'Race', 'SHMostRec', 'SHNumLast12Mon', 'DKAMostRec', 'DKANumLast12Mon', 'OthGlucLowerMed', 'PEAbnormal'], use_cat_names=True)

        # Fit and transform the encoder on the demographic data
        demographics_encoded = encoder.fit_transform(demographics)

        # Merge the encoded demographic data with each dataset
        self.training_set = self.training_set.merge(demographics_encoded, how='left', left_on='id', right_on='PtID')
        self.validation_set = self.validation_set.merge(demographics_encoded, how='left', left_on='id', right_on='PtID')
        self.test_set = self.test_set.merge(demographics_encoded, how='left', left_on='id', right_on='PtID')

    def aggregate_window(self, func: Callable, func_name: str) -> None:
        """
        Apply func over various window sizes and add columns to training and validation sets for the output of these aggregate functions
        
        Args:
        func (Callable): A function to apply to each aggregate window.
        func_name (str): The name of the function, used to create new column names.
        """
        # Define the time intervals in minutes for aggregation
        time_windows = {
            'last_10_minutes': 10,
            'last_30_minutes': 30,
            'last_1_hour': 60,
            'last_3_hours': 180,
            'last_6_hours': 360,
            'last_12_hours': 720
        }

        # Convert the column names to a format we can perform calculations on (number of minutes since 06:00:00)

        stamps = self.training_set.columns[4:220]
        
        times = pd.to_timedelta(stamps).total_seconds()/60  # Convert to minutes

        # Apply the aggregation function over specified time windows
        for window_name, minutes in time_windows.items():
            # Find the time range for each window
            max_time = times.max()
            min_time = max_time - minutes

            # Get columns that fall within the current time window
            columns_to_aggregate = [time for time in stamps if min_time < pd.to_timedelta(time).total_seconds()/60 <= max_time]

            # For FFT, handle real and imaginary parts separately
            if func == self.fft:
                # Convert DataFrames to arrays before applying FFT
                fft_results_train = np.apply_along_axis(func, 1, self.training_set[columns_to_aggregate].values)
                fft_results_valid = np.apply_along_axis(func, 1, self.validation_set[columns_to_aggregate].values)
                fft_results_test = np.apply_along_axis(func, 1, self.test_set[columns_to_aggregate].values)

                for i in range(fft_results_train.shape[1]):
                    self.training_set[f'{func_name}_{window_name}_real_{i}'] = fft_results_train[:, i].real
                    self.training_set[f'{func_name}_{window_name}_imag_{i}'] = fft_results_train[:, i].imag
                    self.validation_set[f'{func_name}_{window_name}_real_{i}'] = fft_results_valid[:, i].real
                    self.validation_set[f'{func_name}_{window_name}_imag_{i}'] = fft_results_valid[:, i].imag
                    self.test_set[f'{func_name}_{window_name}_real_{i}'] = fft_results_test[:, i].real
                    self.test_set[f'{func_name}_{window_name}_imag_{i}'] = fft_results_test[:, i].imag
            
            else:
                # Apply the function to the selected columns and store in a new column
                self.training_set[f'{func_name}_{window_name}'] = self.training_set[columns_to_aggregate].apply(func, axis=1)
                self.validation_set[f'{func_name}_{window_name}'] = self.validation_set[columns_to_aggregate].apply(func, axis=1)
                self.test_set[f'{func_name}_{window_name}'] = self.test_set[columns_to_aggregate].apply(func, axis=1)



In [150]:
test = FeatureEngine()

/var/folders/6j/zxy_nmpx04gg6ssx89r9gl5h0000gn/T/ipykernel_724/201553568.py:3: DtypeWarning: Columns (220) have mixed types. Specify dtype option on import or set low_memory=False.
  self.training_set = pd.read_csv("cleaned_data/train_data.csv")


In [151]:
test.test_set.head()

,Unnamed: 0,index,id,corresponding_day,06:00:00,06:05:00,06:10:00,06:15:00,06:20:00,06:25:00,...,23:45:00,23:50:00,23:55:00,hypo,tar_last_10_minutes,tar_last_30_minutes,tar_last_1_hour,tar_last_3_hours,tar_last_6_hours,tar_last_12_hours
0,1514,1514,15,2015-04-23,125.0,134.0,131.0,142.0,157.0,169.0,...,186.0,180.0,173.0,False,0.0,0.666667,0.833333,0.944444,0.638889,0.569444
1,1515,1515,15,2015-04-24,154.0,156.0,159.0,158.0,161.0,143.0,...,165.0,166.0,165.0,True,0.0,0.000000,0.000000,0.527778,0.527778,0.263889
2,1516,1516,15,2015-04-25,115.0,108.0,96.0,86.0,82.0,87.0,...,162.0,165.0,167.0,True,0.0,0.000000,0.000000,0.000000,0.069444,0.125000
3,1517,1517,15,2015-04-26,227.0,228.0,229.0,231.0,240.0,244.0,...,322.0,326.0,316.0,True,1.0,1.000000,1.000000,1.000000,0.680556,0.625000
4,1518,1518,15,2015-04-27,172.0,181.0,178.0,172.0,189.0,195.0,...,126.0,123.0,115.0,False,0.0,0.000000,0.000000,0.583333,0.444444,0.534722


In [92]:
"""
TODO:
2. Add Morlet Mexican Hat Columns
"""

''